In [2]:
import numpy as np
import json
import tensorflow as tf
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

2022-06-24 08:46:11.749573: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-24 08:46:11.749602: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
directory = "audioset_v1_embeddings/eval"
dataset = []
for file_name in os.listdir(directory):
     if file_name.endswith(".tfrecord"):
            dataset.append(os.path.join(directory,file_name))

In [4]:
tf.compat.v1.enable_eager_execution()

In [5]:
raw_dataset = tf.data.TFRecordDataset(dataset)

2022-06-24 08:49:52.641696: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-24 08:49:52.641732: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-24 08:49:52.641759: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debian): /proc/driver/nvidia/version does not exist
2022-06-24 08:49:52.644310: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
class_labels = pd.read_csv('class_labels_indices.csv')
labels = class_labels['display_name'].tolist()
music_class = class_labels[class_labels['display_name'].str.contains('Music', case=False)]
music_labels = music_class['index'].tolist()

In [7]:
audios = []
NUM_SECONDS = 10
for raw_record in raw_dataset: # Load 2,000 files
    example = tf.train.SequenceExample()
    example.ParseFromString(raw_record.numpy())
    # Metadata of samples
    audio_labels = example.context.feature['labels'].int64_list.value
    start_time = example.context.feature['start_time_seconds'].float_list.value
    end_time = example.context.feature['end_time_seconds'].float_list.value
    video_id = example.context.feature['video_id'].bytes_list.value
    if not (set(music_labels) & set(audio_labels)):
        continue
    # Feature
    feature_list = example.feature_lists.feature_list['audio_embedding'].feature
    final_features = [list(feature.bytes_list.value[0]) for feature in feature_list]
    audio_embedding = [item for sublist in final_features[:NUM_SECONDS] for item in sublist]
    if len(final_features) < NUM_SECONDS:
        continue
    audio = {
        'label': audio_labels, 
        'video_id': video_id[0], 
        'start_time': start_time[0], 
        'end_time': end_time[0],
        'data': audio_embedding
    }
    audios.append(audio)

In [17]:
with open('music_set.json', 'w') as file:
    str_audio = repr(audios)
    json.dump(str_audio, file)